<a class="anchor" id="toc-go-back"></a>
# Assignment 3: Day-ahead scheduling from the perspective of the system operator

### Table of Contents
* [1. Loading the data](#data-loading)
* [2. Setup optimization problem](#optimziation-problem-setup)
* [3. Running the optimization model ](#run-optimization)
* [4. Can we solve the problem using machine learning?](#machine-learning)
* [5. Extra ](#extra)

First of all, we refer to the hand-in for an in-depth description of the problem formulation and the optimization problem. This notebooks focuses on implementing the model using ```Gurobipy``` and an additional part using machine learning. 

We start by loading the relevant modules.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import gurobipy as grb

In [2]:
DATA_DIR = Path('../data')

<a class="anchor" id="data-loading"></a>
## 1. Loading the data

We initially load the data files given to us with the assignment description. This will later on be used as input to the optimization model from which we can then find the optimal values of the decision variables.

In [3]:
# Load data files from csv-files
pmax    = pd.read_csv(DATA_DIR / 'pgmax.csv')
pmin    = pd.read_csv(DATA_DIR / 'pgmin.csv')
ru      = pd.read_csv(DATA_DIR / 'ramp.csv')
UT      = pd.read_csv(DATA_DIR / 'lu.csv')
DT      = pd.read_csv(DATA_DIR / 'ld.csv')    
demand  = pd.read_csv(DATA_DIR / 'Demand.csv', sep=';')   
c_op    = pd.read_csv(DATA_DIR / 'cost_op.csv') 
c_st    = pd.read_csv(DATA_DIR / 'cost_st.csv') 
PTDF    = pd.read_csv(DATA_DIR / 'PTDF.csv', sep=';') 
busgen  = pd.read_csv(DATA_DIR / 'busgen.csv', sep=';')
busload = pd.read_csv(DATA_DIR / 'busload.csv', sep=';')
fmax    = pd.read_csv(DATA_DIR / 'fmax.csv')

# ???????????????????????????????????????????????????????????????????????????
Hg      = pd.DataFrame(np.dot(PTDF, busgen), index=[f'Line {i+1}' for i in range(PTDF.shape[0])], columns=[f'Gen {i+1}' for i in range(busgen.shape[1])])
Hl      = pd.DataFrame(np.dot(PTDF, busload), index=[f'Line {i+1}' for i in range(PTDF.shape[0])], columns=[f'Load {i+1}' for i in range(busload.shape[1])])

With the data set loaded, we can now define the basics underlying the optimization problem - thus, we start by defining the numerical values of the parameters.

In [4]:
N_g     = busgen.shape[1]   # the number of generator units
N_t     = demand.shape[0]   # next 24 hours
N_load  = busload.shape[1]  # the number of load buses
N_lines = PTDF.shape[0]     # the number of transmission lines

In [113]:
N_g     = 5
N_t     = 24
N_load  = 8
N_lines = 16

<a class="anchor" id="optimization-problem-setup"></a>
## 2. Setup optimization problem

Differences between the hand-in and the proposed formula from class:
1. start-up cost is deterministic and should not be optimized for (i.e. input parameter and not decision variable)
2. demand is additionally indexed by time for the flow constraint
3. the start-up constraint is added by specifying an on/off variable for each unit at each time, $b$.

In [115]:
model = grb.Model()

Now, we are ready to define the decision variables of the optimization problem. These relate to the start-up status and production of generator units at each time step of the day. We additionally include a variable for denoting the on/off status. As we are given a data file for start-up costs for each generation unit, we approach the problem with the start-up cost being time-independent and consider it as an input rather than a decision variable.

In [116]:
# Add variables for each generator unit at each time step for specifying on/off and start-up status
b   = model.addVars(N_g, N_t, vtype=grb.GRB.BINARY)
u   = model.addVars(N_g, N_t, vtype=grb.GRB.BINARY)

# Add variable denoting the power output of each generator unit at each time step
p   = model.addVars(N_g, N_t, vtype=grb.GRB.CONTINUOUS)

# We add slack variables for ensuring feasability of the power balance equations
eps     = model.addVars(N_t, 1, vtype=grb.GRB.CONTINUOUS)
delta   = model.addVars(N_t, 1, vtype=grb.GRB.CONTINUOUS)

# Make variable updates effective
model.update()

Next, we add the constraints to the optimization model. Overall these concern:
1) Power balance equation, i.e. the total production must equal the total demand. Here, we add the concept of slack variable for feasibility - i.e. making sure that the problem can be solved even if in some cases the demand is not met.
2) Generation limits, ensuring that the generator units do not produce more than their maximum capacity.
3) Ramp-up and ramp-down constraints, ensuring...
4) Line technical limits, ensuring that the flow between busses does not exceed the maximum capacity of the line.
5) Minimum on/off times.
6) Start-up constraints for turning on a generator unit (i.e. the unit must be off in the previous time step).

In [117]:
d_vector = demand.iloc[0, :]

In [118]:
from tqdm import tqdm

for t in tqdm(range(N_t), desc='Adding constraints...'):

    ### POWER BALANCE EQUATION ###
    # Add power balance constraints for each time step
    model.addConstr(sum(p[g_n, t] for g_n in range(N_g)) == sum(demand.iloc[t, n] for n in range(N_load)) + eps[t, 0] - delta[t, 0])
    
    # Add positive constraints for the slack variables
    model.addConstr(eps[t, 0] >= 0)
    model.addConstr(delta[t, 0] >= 0)

    for g in range(N_g):
        
        ### GENERATION LIMITS ###
        model.addConstr(b[g, t] * pmin.iloc[g, 0] <= p[g, t])
        model.addConstr(b[g, t] * pmax.iloc[g, 0] >= p[g, t])
        
        if t > 1:
            ### GENERATOR ON/OFF AND START-UP STATUS ###
            model.addConstr(u[g, t] >= b[g, t] - b[g, t-1])

            ### RAMPING CONSTRAINTS ###
            model.addConstr(p[g, t] - p[g, t-1] <= ru.iloc[g, 0])
            model.addConstr(p[g, t-1] - p[g, t] <= ru.iloc[g, 0])

            ### MINIMUM ON TIME ###
            min_on_time_generator = min(UT.iloc[g, 0] + t - 1, N_t)
            for tau in range(t, min_on_time_generator):
                model.addConstr(-b[g, t-1] + b[g, t] - b[g, tau] <= 0)
            
            ### MINIMUM OFF TIME ###
            min_off_time_generator = min(DT.iloc[g, 0] + t - 1, N_t)
            for tau in range(t, min_off_time_generator):
                model.addConstr(b[g, t-1] - b[g, t] + b[g, tau] <= 1)
                        
    ### LINE FLOW LIMITS ###
    for l in range(N_lines):

        # NOTE: should we index time step for indexing the demand? it is not super obvious from the definition to me
        # line_expr = sum(Hg.iloc[l, g_n] * p[g_n, t] for g_n in range(N_g)) - sum(Hl.iloc[l, n] * demand.iloc[t, n] for n in range(N_load)) - eps[t, 0] + delta[t, 0]
        line_expr = sum(Hg.iloc[l, g_n] * p[g_n, t] for g_n in range(N_g)) - sum(Hl.iloc[l, n] * d_vector.iloc[n] for n in range(N_load)) - eps[t, 0] + delta[t, 0]
        model.addConstr(line_expr <= fmax.iloc[l, 0])
        model.addConstr(line_expr >= -fmax.iloc[l, 0]) #NOTE: is this relevant?
    
model.update()

Adding constraints...:   0%|          | 0/24 [00:00<?, ?it/s]

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


Adding constraints...: 100%|██████████| 24/24 [00:00<00:00, 69.74it/s]


Lastly, we define the objective function for the optimization problem. This is defined as minimizing the total cost - i.e. production cost and start-up cost. In this case, we assume deterministic start-up costs for each generator unit - in other words, we assume that they are time-independent.

In [119]:
# QUESTION: do we need the start-up cost constraint (i.e. s_gt) when having it as an input variable?
# QUESTION: what is the M variable in the modified cost function? 
# QUESTION: what is the decision variable q denoting? for the modified cost function?

In [120]:
# Set minimization objective function
minimum_objective   = sum(c_op.iloc[g, 0] * p[g, t] + c_st.iloc[g, 0] * u[g, t] for g in range(N_g) for t in range(N_t))
# Set slack objective function
M = 1
slack_objective     = M * sum(eps[t, 0] + delta[t, 0] for t in range(N_t))

# Set objective function
model.setObjective(sense=grb.GRB.MINIMIZE, expr=minimum_objective + slack_objective)
model.update()

<a class="anchor" id="run-optimization"></a>
## 3. Running the optimization model

In [121]:
model.getConstrs().__len__()

1942

In [122]:
opt = model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11375H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1942 rows, 408 columns and 7878 nonzeros
Model fingerprint: 0xdaec7d34
Variable types: 168 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [5e-03, 3e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 92721.290796
Presolve removed 689 rows and 22 columns
Presolve time: 0.05s
Presolved: 1253 rows, 386 columns, 4207 nonzeros
Found heuristic solution: objective 90211.915608
Variable types: 161 continuous, 225 integer (225 binary)

Root relaxation: objective 4.458406e+04, 85 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [133]:
sum(p[g, t].x * c_op.iloc[g, 0] + u[g, t].x * c_st.iloc[g, 0] for g in range(N_g) for t in range(N_t))

43052.561275546745

In [137]:
for t in range(24):
    for g in range(N_g):
        print(b[g, t].x)

0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0


In [132]:
for t in range(24):
    for g in range(N_g):
        print(p[g, t].x)

0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
127.63524592426413
0.0
0.0
0.0
0.0
155.59925291162497
0.0
0.0
0.0
0.0
164.3444207159024
0.0
0.0
0.0
0.0
165.98127565948744
0.0
0.0
0.0
0.0
164.60195510134267
0.0
0.0
0.0
0.0
163.78352762955015
0.0
0.0
0.0
0.0
162.1466726859651
0.0
0.0
0.0
0.0
156.93274915429794
0.0
0.0
0.0
0.0
159.13049718423528
0.0
0.0
0.0
0.0
173.3929472210919
0.0
0.0
0.0
0.0
182.95654249716182
0.0
0.0
0.0
0.0
182.13811502536925
0.0
0.0
0.0
0.0
171.710267962035
0.0
0.0
0.0
0.0
158.2662453969709
0.0
0.0
0.0
0.0
139.0474062138874
0.0
0.0
0.0
0.0
112.97778855555165
0.0
0.0
0.0
0.0
100.0


<a class="anchor" id="machine-learning"></a>
## 4. Can we solve the problem using machine learning?

<a class="anchor" id="extra"></a>
## 5. Extra